# I am trying to understand the methods of the public notebooks and note in this notebook.

I am a beginner for the Recommended system.

**I hope it is helpful for u.**

**Continuous update!!!**

**First all. I use the sample of the all data base on  https://www.kaggle.com/paweljankiewicz/hm-create-dataset-samples.**

**It sample 0.1%, 1%, 5% of the all data. It save a lot of time when u are learning.**

You can use it directly. https://www.kaggle.com/datasets/hjimbean/hm-samples

# 1. 【LB: 0.0220】 base on https://www.kaggle.com/code/hengzheng/time-is-our-best-friend-v2/notebook

## 1.1 data processing

It only use the article_id in transactions_train.csv. It dont use the detail information like the images of goods, the customer age or artcles detail something...

### This method only uses data from the last three weeks.

transactions_1w is the data from 2020-09-15 to 2020-09-22

transactions_2w is the data from 2020-09-07 to 2020-09-14

transactions_3w is the data from 2020-08-31 to 2020-09-06

In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path

# data_path = Path('/kaggle/input/h-and-m-personalized-fashion-recommendations/')
data_path = Path('../input/hm-samples/')

transactions = pd.read_csv(
    data_path / 'transactions_train_sample1.csv',
    # set dtype or pandas will drop the leading '0' and convert to int
    dtype={'article_id': str} 
)

submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])

transactions_3w = transactions[transactions['t_dat'] >= pd.to_datetime('2020-08-31')].copy()
transactions_2w = transactions[transactions['t_dat'] >= pd.to_datetime('2020-09-07')].copy()
transactions_1w = transactions[transactions['t_dat'] >= pd.to_datetime('2020-09-15')].copy()

### the shape of data is same with transactions_train.csv

In [ ]:
transactions.head()

In [ ]:
transactions_1w.head()

### turn this data into a dict，like：{customer_id: {article_id: count}}

count: this week the same customer_id buy how many times of the same article_id

In [ ]:
purchase_dict_3w = {}

for i,x in enumerate(zip(transactions_3w['customer_id'], transactions_3w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_3w:
        purchase_dict_3w[cust_id] = {}
    
    if art_id not in purchase_dict_3w[cust_id]:
        purchase_dict_3w[cust_id][art_id] = 0
    
    purchase_dict_3w[cust_id][art_id] += 1
    
print(len(purchase_dict_3w))

dummy_list_3w = list((transactions_3w['article_id'].value_counts()).index)[:12]

purchase_dict_2w = {}

for i,x in enumerate(zip(transactions_2w['customer_id'], transactions_2w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_2w:
        purchase_dict_2w[cust_id] = {}
    
    if art_id not in purchase_dict_2w[cust_id]:
        purchase_dict_2w[cust_id][art_id] = 0
    
    purchase_dict_2w[cust_id][art_id] += 1
    
print(len(purchase_dict_2w))

dummy_list_2w = list((transactions_2w['article_id'].value_counts()).index)[:12]


purchase_dict_1w = {}

for i,x in enumerate(zip(transactions_1w['customer_id'], transactions_1w['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict_1w:
        purchase_dict_1w[cust_id] = {}
    
    if art_id not in purchase_dict_1w[cust_id]:
        purchase_dict_1w[cust_id][art_id] = 0
    
    purchase_dict_1w[cust_id][art_id] += 1
    
print(len(purchase_dict_1w))

dummy_list_1w = list((transactions_1w['article_id'].value_counts()).index)[:12]

**dummy_list_3w, purchase_dict_2w, purchase_dict_1w** is Top 12 Most Popular Items of the week.

In [ ]:
purchase_dict_3w['045fc8403924222479a2263f84258b9bf7b02cb6ddfe2d495bd8a43583165981']

## 1.2 predict

If you purchased something in the last week, it will be sorted and recommended according to the number of items you purchased. If you purchased less than 12 items, it will be recommended and supplemented according to the most popular items in the last week, and the data of the following weeks will not be seen. 

If you didn't buy anything in the last week, look at the last two weeks, then look at the last three weeks. If you haven't bought anything at all, it will recommend you the most popular products in the last week.

In [ ]:
not_so_fancy_but_fast_benchmark = submission[['customer_id']]
prediction_list = []

dummy_list = list((transactions_1w['article_id'].value_counts()).index)[:12]
dummy_pred = ' '.join(dummy_list)

for i, cust_id in enumerate(submission['customer_id'].values.reshape((-1,))):
    if cust_id in purchase_dict_1w:
        l = sorted((purchase_dict_1w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = ' '.join(l[:12])
        else:
            s = ' '.join(l+dummy_list_1w[:(12-len(l))])
    elif cust_id in purchase_dict_2w:
        l = sorted((purchase_dict_2w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = ' '.join(l[:12])
        else:
            s = ' '.join(l+dummy_list_2w[:(12-len(l))])
    elif cust_id in purchase_dict_3w:
        l = sorted((purchase_dict_3w[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = ' '.join(l[:12])
        else:
            s = ' '.join(l+dummy_list_3w[:(12-len(l))])
    else:
        s = dummy_pred
    prediction_list.append(s)

not_so_fancy_but_fast_benchmark['prediction'] = prediction_list
print(not_so_fancy_but_fast_benchmark.shape)
not_so_fancy_but_fast_benchmark.head()